In [1]:
from ortools.linear_solver import pywraplp


In [2]:
N = 13
M = 3
X = []
tc_inp = [[0, 2, 3, 4, 8, 10], [0, 1, 3, 5, 6, 7, 8], [1, 2, 3, 7, 9, 11, 12]]
tc = [[0 for i in range(M)] for j in range(N)]
for i in range(M):
    for item in tc_inp[i]:
        tc[item][i] = 1

f_inp = '0 2,0 4,0 8,1 4,1 10,3 7,3 9,5 11,5 12,6 8,6 12'
f = [[0 for i in range(N)] for j in range(N)]
for i in f_inp.split(','):
    i_1, i_2 = i.split()
    f[int(i_1)][int(i_2)] = 1
    f[int(i_2)][int(i_1)] = 1

c = [3, 3, 4, 3, 4, 3, 3, 3, 4, 3, 3, 4, 4]

In [3]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

infinity = solver.infinity()

for i in range(N):
    X_i = []
    for j in range(M):
        x = solver.IntVar(0, 1, 'x({},{})'.format(i, j))
        X_i.append(x)
    X.append(X_i)

y = solver.IntVar(0, infinity, 'y')

for i in range(N):
    s = 0
    for j in range(M):
        solver.Add(tc[i][j] - X[i][j] >= 0)
        t = 0
        for i_1 in range(N):
            solver.Add(f[i][i_1]*(X[i][j] + X[i_1][j]) <= 1)
            t += X[i_1][j]*c[i_1]

        solver.Add(t - y <= 0)

        s += X[i][j]

    solver.Add(s == 1)


print('Number of variables =', solver.NumVariables())

print('Number of constraints =', solver.NumConstraints())

solver.Minimize(y)

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    for j in range(M):
        res_k = []
        for i in range(N):
            if X[i][j].solution_value() == 1:
                res_k.append(i)
        print(res_k)


else:
    print('The problem does not have an optimal solution.')

print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations' % solver.iterations())
print('Problem solved in %d branch-and-bound nodes' % solver.nodes())


Number of variables = 40
Number of constraints = 598
Solution:
Objective value = 14.999999999999998
[3, 4, 8, 10]
[0, 1, 5, 6, 7]
[2, 9, 11, 12]

Advanced usage:
Problem solved in 39.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 1 branch-and-bound nodes
